In [ ]:
import datetime
import glob
import matplotlib.pyplot as plt
import numpy as np
import warnings

import neuralpde



warnings.warn('Whatever this is :(')
neuralpde.network.DEVICE = 'cpu'



subset_xy = np.s_[174:235, 64:110]
"""
Subset of (x, y) grid to use by index, ordered as (+/- longitude, +/- latitude)

# WARNING
Ordering here is not what you expect!  Be careful to check!
"""

q = 10
maps = 7


Alright, let's load some data and do some preliminary checks:

In [ ]:
d = neuralpde.nc.SeaIceV4(sorted(glob.glob('data/V4/*197*.nc')))

day = np.searchsorted(d.date, datetime.date(1979, 8, 15))
indices = np.arange(day - maps, day + maps + 1)

neuralpde.nc.check_boundaries(indices, d)


Do some preliminary prep on the data:

In [ ]:
u = d.seaice_conc[indices, *subset_xy]
u[np.isnan(u)] = 0.  # mask out NaN

(scalex, scaley), (x, y) = neuralpde.network.normalize_xy(d.meters_x, d.meters_y)
x, y = np.meshgrid(x, y)  # this ordering is annoying but necessary
x, y = x[subset_xy], y[subset_xy]

mask_coast = (d.flag_coast)[day, *subset_xy]
mask_other = (d.flag_land | d.flag_hole | d.flag_lake | d.flag_missing)[day, *subset_xy]
mask_any = mask_coast | mask_other


Now we define the loss hyperparameters with respect to which we will train:

In [ ]:
# loss hyperparameters (i.e., relative weighting of terms in loss)
weights = np.array(
    [
        10.,  # differential loss (t_{n})
        10.,  # differential loss (t_{n+1})
        1.,  # boundary loss, no slip + no pen
        1.,  # kappa regularization
        1.,  # v regularization
        5.,  # f minimization
    ]
)
weights = weights / np.sqrt(np.sum(weights**2))


Now do the training.  :)

In [ ]:
net = neuralpde.network.Network(q = q, shape = u.shape, kernel = 5).to(neuralpde.network.DEVICE, neuralpde.network.DTYPE)
losses = net.fit(x, y, u, weights, mask_coast, mask_other, epochs=1000, lr=1e-3)


In [ ]:
plt.plot(losses)
plt.yscale('log')


In [ ]:
outs = net.predict(x, y, u)
kappa = outs[0]
v1 = outs[1]
v2 = outs[2]
f = outs[3]



cmap = plt.get_cmap('Blues_r')
cmap.set_bad(color='darkgreen', alpha=0.6)
cmap.set_over(color='violet', alpha=0.4)
cmap.set_under(color='cyan', alpha=0.4)

mask_any_plotting = np.ones_like(mask_any, dtype=float)
mask_any_plotting[mask_any] = np.nan



plt.pcolormesh(x, y, d.seaice_conc[day, *subset_xy], cmap=cmap, vmin=0, vmax=1, shading='nearest')
plt.colorbar()
plt.gca().set_aspect('equal')
plt.show()

plt.pcolormesh(x, y, (outs[4] * mask_any_plotting), cmap=cmap, vmin=0, vmax=1)
plt.colorbar()
plt.gca().set_aspect('equal')
plt.show()

plt.pcolormesh(x, y, (kappa * mask_any_plotting))
plt.colorbar()
plt.gca().set_aspect('equal')
plt.show()

plt.pcolormesh(x, y, (v1 * mask_any_plotting))
plt.colorbar()
plt.gca().set_aspect('equal')
plt.show()

plt.pcolormesh(x, y, (v2 * mask_any_plotting))
plt.colorbar()
plt.gca().set_aspect('equal')
plt.show()

plt.pcolormesh(x, y, (f * mask_any_plotting))
plt.colorbar()
plt.gca().set_aspect('equal')
plt.show()
